# 16s rRNA Microbiome Analysis Pipeline Using QIIME2

This pipeline uses QIIME2 to analyse 16S rRNA gene amplicon single end or paired end sequences to produce taxonomy plots and feature tables to analyse the microbiome of the sample. Data can then be exported and analysed on external microbiome analytic tools, such as STAMP.

### Set up sample

Data will be manipulated so that it can be registered by QIIME2. Set up sample-metadata using Keemei beforehand to validate the tabular bioinformatics file format.

#### If Single end...

#### Extract barcodes from fastq and create barcode.fastq

In [ ]:
%%bash
/home/single_barcode_cut [seqname].fastq

#### Remove barcodes from fastq and create sequences.fastq

In [ ]:
%%bash
/home/single_barcode_remove [seqname].fastq

#### Gzip fastq files

In [23]:
%%bash
gzip sequences.fastq
gzip barcodes.fastq

#### If Paired end...

Rename forward and reverse sequences to og_forward.fastq and reverse.fastq respectively

#### Create barcodes.fastq from og_forward.fastq

In [8]:
%%bash
/home/barcode_cut.py og_forward.fastq

Traceback (most recent call last):
  File "/home/aran/Desktop/Project/barcode_cut.py", line 2, in <module>
    with open("og_forward.fastq", "r") as data_in, open("barcodes.fastq", "w") as data_out:
IOError: [Errno 2] No such file or directory: 'og_forward.fastq'


CalledProcessError: Command 'b'/home/aran/Desktop/Project/barcode_cut.py og_forward.fastq\n'' returned non-zero exit status 1.

#### Remove barcodes from og_forward.fastq and create forward.fastq

In [ ]:
%%bash
/home/barcode_remove og_forward.fastq

#### Gzip forward.fastq, reverse.fastq and barcode.fastq

In [ ]:
%%bash
gzip forward.fastq
gzip reverse.fastq
gzip barcodes.fastq

### Install latest version of QIIME2

In [1]:
#conda update conda
#conda install wget
#wget https://data.qiime2.org/distro/core/qiime2-2020.6-py36-linux-conda.yml
#conda env create -n qiime2-2020.6 --file qiime2-2020.6-py36-linux-conda.yml
##Cleanup
#rm qiime2-2020.6-py36-linux-conda.yml

#### Activate conda environmemt and insure installation worked

In [2]:
%%bash
conda activate qiime2-2020.6
qiime --help

SyntaxError: invalid syntax (<ipython-input-2-08af0443c59f>, line 1)

### Move and rename sequences

Manipulated data will now be converted into a .qza folder, which will include the sequence(s) and barcode fastq.gz

If single end...

In [1]:
import shutil
mkdir emp-single-end-sequences
sequences = '/home/sequences.fastq.gz'
barcode = '/home/barcodes.fastq.gz'
new_path = '/home/emp-single-end-sequences'
shutil.move(sequences, new_path)
shutil.move(barcode, new_path)

mkdir: cannot create directory ‘emp-single-end-sequences’: File exists
bash: line 2: fg: no job control
mv: cannot stat 'sequences.fastqc.gz': No such file or directory
bash: line 4: fg: no job control
mv: cannot stat 'barcodes.fastqc.gz': No such file or directory


CalledProcessError: Command 'b'mkdir emp-single-end-sequences\n%%bash\nmv sequences.fastqc.gz /home/emp-single-end-sequences\n%%bash\nmv barcodes.fastqc.gz /home/emp-single-end-sequences\n'' returned non-zero exit status 1.

In [25]:
%%bash
qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza

Imported emp-single-end-sequences as EMPSingleEndDirFmt to emp-single-end-sequences.qza


If paired end...

In [20]:
mkdir emp-paired-end-sequences
forward = '/home/aran/Desktop/Project/forward.fastq.gz'
reverse = '/home/reverse.fastq.gz'
barcode = '/home/barcodes.fastq.gz'
new_path = '/home/emp-paired-end-sequences'
shutil.move(forward, new_path)
shutil.move(reverse, new_path)
shutil.move(barcode, new_path)

['alpha-rarefaction.qzv', 'sample-metadata.tsv', 'emp-single-end-sequences.qza', 'rooted-tree.qza', 'Untitled.ipynb', 'masked-aligned-rep-seqs.qza', '.barcode_cut.py.swp', 'unrooted-tree.qza', 'aligned-rep-seqs.qza', 'demux.qza', '.ipynb_checkpoints', 'rep-seqs.qzv', 'Metagenomics Analysis Pipeline.ipynb', 'core-metrics-results', 'gg-13-8-99-515-806-nb-classifier.qza', 'table.qza', 'demux-details.qza', 'emp-single-end-sequences', 'stats-dada2.qzv', 'taxa-bar-plots.qzv', 'table.qzv', 'rep-seqs.qza', 'single_barcode_remove.py', 'barcode_remove.py', 'barcode_cut.py', 'taxonomy.qza', 'emp-paired-end-sequences', 'stats-dada2.qza', 'Practice.ipynb', 'barcode_extract.sh.save', 'single_barcode_cut.py', 'taxonomy.qzv']


In [19]:
%%bash
qiime tools import \
  --type EMPPairedEndSequences \
  --input-path emp-paired-end-sequences \
  --output-path emp-paired-end-sequences.qza

bash: line 1: qiime: command not found


CalledProcessError: Command 'b'qiime tools import \\\n  --type EMPPairedEndSequences \\\n  --input-path emp-paired-end-sequences \\\n  --output-path emp-paired-end-sequences.qza\n'' returned non-zero exit status 127.

### Demultiplexing sequences

Demultiplex sequence using the barcode information from the sample-metadata.tsv to determine sequences. This barcode used in the sample is expected to follow the Earth Microbiome Project protocol.

The first output 'demux.qza' will contain the demultiplexed sequences.

The second output 'demux-details.qza' will contain Golay error correction details

#### Single end

In [26]:
%%bash
qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences demux.qza \
  --o-error-correction-details demux-details.qza

Saved SampleData[SequencesWithQuality] to: demux.qza
Saved ErrorCorrectionDetails to: demux-details.qza


#### Paired end

In [16]:
%%bash
qiime demux emp-paired \
    --i-seqs emp-paired-end-sequences.qza \
    --m-barcodes-file sample-metadata.tsv \
    --m-barcodes-column barcode-sequence \
    --o-per-sample-sequences demux.qza \
    --o-error-correction-details demux-details.qza

Plugin error from demux:

  No sequences were mapped to samples. Check that your barcodes are in the correct orientation (see the rev_comp_barcodes and/or rev_comp_mapping_barcodes options). If barcodes are NOT Golay format set golay_error_correction to False.

Debug info has been saved to /tmp/qiime2-q2cli-err-dle9rohq.log


CalledProcessError: Command 'b'qiime demux emp-paired \\\n    --i-seqs emp-paired-end-sequences.qza \\\n    --m-barcodes-file sample-metadata.tsv \\\n    --m-barcodes-column barcode-sequence \\\n    --o-per-sample-sequences demux.qza \\\n    --o-error-correction-details demux-details.qza\n'' returned non-zero exit status 1.

#### Demux summary

Generate a summary of the demultiplexing results to observe number of sequences per sample and obsereve sequence quality

In [ ]:
%%bash
qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

### Sequence quality control and feature table construction

QC data using DADA2 - used to remove markers and filter chimeric sequences.

--p-trim-left m: trims first m bases

--p-trunc-len n: truncates to n size

Observe Interactive Quality Plot tab in the demux.qzv to determine m and n. 

The first output 'rep-seqs.qza' will contain a FeatureData[Sequence] used to map feature identifiers in the FeatureTable to the sequences they represent.

The second output 'table.qza' will contain a frequency table of each unique sequence in each sample in the dataset.

The third output 'stats-dada2.qza' will have a statistical summary of the denoising.

#### Single end

In [27]:
%%bash
qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 120 \
  --o-representative-sequences rep-seqs.qza \
  --o-table table.qza \
  --o-denoising-stats stats-dada2.qza

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: stats-dada2.qza


#### Paired end

Added forward and reverse trim and trunc functions

In [ ]:
%%bash
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left-f [m] \
  --p-trim-left-r [m] \
  --p-trunc-l-len [n] \
  --p-trunc-r-len [n] \
  --o-representative-sequences rep-seqs.qza \
  --o-table table.qza \
  --o-denoising-stats stats-dada2.qza

#### Generate table of statistics

In [28]:
%%bash
qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv

Saved Visualization to: stats-dada2.qzv


### FeatureTable and FeatureData summaries

This step will be used on the outputs rep-seqs.qza and table.qza to create visual summaries of the QC'd data within these .qza folders.

feature-table tabulate-seqs used to provide links to easily BLAST each sequence against the NCBI nt database as wel as to create histograms of distrabutions and summary statistics.

In [30]:
%%bash
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv


### Generate a tree for phylogenetic diversity analyses

The mafft program is used to establish multiple sequence alignment of the sequences from FeatureData[Sequence] and filters positions that are highly variable. FastTree is performed to create a phylogenetic tree from the masked alignment.

The first output 'aligned-rep-seqs.qza' will create FeatureData[AlignedSequence] from FeatureData[Sequence].

The second output 'masked-aligned-rep-seqs.qza' will create FeatureData[AlignedSequence] which removes positions that are highly variable.

The third output 'unrooted-tree.qza' will create an unrooted phylogenetic tree from the masked FeatureData[AlignedSequence].

The fourth output 'rooted-tree.qza' will create an rooted phylogenetic tree from the masked FeatureData[AlignedSequence].

In [31]:
%%bash
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


### Alpha and beta diversity analysis

This next step will generate numerous outputs to allow for Alpha and beta diversity analysis. The main outputs generated are:

Alpha diversity:

    Shannon’s diversity index
    Observed Features
    Faith’s Phylogenetic Diversity
    Pielou’s Evenness
    
Beta diversity:

    Jaccard distance
    Bray-Curtis distance
    unweighted UniFrac distance
    weighted UniFrac distance
    
The --p-sampling-depth p parameter is observed from the table.qzv file from above. This parameter is important as it will remove any sequences after being sub-sampled that are smaller than lenght p. This will result in data being excluded from taxanomic analysis, therefore it is best to obtain the best value for p to not exclude too much information from your analysis.
    

In [32]:
%%bash
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 1103 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

Check for any links between categorical metadata columns and alpha diversity data using Faith Phylogenetic Diversity, a qualitiative measure of community richness which includes phylogenetic relationships between the features, and Pielou’s Evenness, which measures community evenness.

In [33]:
%%bash
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv
Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


Check with unweighted_unifrac_distance_matrix.qza if distances between samples within a group are more similar to each other then they are to samples from the other groups using the --p-pairwise command. This command is based on permutation tests so it is best to reference specific columns in the metadata, i.e body-site and subject from sample metadata.

In [34]:
%%bash
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column body-site \
  --o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-body-site-significance.qzv
Saved Visualization to: core-metrics-results/unweighted-unifrac-subject-group-significance.qzv


Generate PCoA using Emperor to observe ordination of microbial community composition using unweighted_unifrac_pcoa_results.qza and bray_curtis_pcoa_results.qza and allow for the option to use time as an observation using --p-custom-axes 

In [35]:
%%bash
qiime emperor plot \
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv

qiime emperor plot \
  --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/bray-curtis-emperor-days-since-experiment-start.qzv

Saved Visualization to: core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv
Saved Visualization to: core-metrics-results/bray-curtis-emperor-days-since-experiment-start.qzv


### Alpha rarefaction plotting

Create two plots from rooted-tree.qza. An alpha rarefaction plot used to determine if the richness of the samples has been fully observed and a visulization that shows the number of samples that remain in each group when the feature table is rarefied to each sampling depth.

Examine Frequency per sample within table.qzv to obtain p value for --p-max-depth

In [37]:
%%bash
qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 4000 \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization alpha-rarefaction.qzv

Saved Visualization to: alpha-rarefaction.qzv


### Taxonomic analysis

Assign taxonomy to the sequences from the FeatureData[Sequence] within rep-seqs.qza using the classifier gg-13-8-99-515-806-nb-classifier.qza. This classifier uses Greengenes 13_8 99% OTUs. This generates a visualisation of the mapping from sequence to taxonomy.

Download Greengenes 13_8 99% OTU classifier
https://data.qiime2.org/2020.6/common/gg-13-8-99-515-806-nb-classifier.qza

In [39]:
%%bash
qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved FeatureData[Taxonomy] to: taxonomy.qza
Saved Visualization to: taxonomy.qzv


Generate interactive barplots to observe taxonomic composition.

In [40]:
%%bash
qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

Saved Visualization to: taxa-bar-plots.qzv
